In [35]:
import refexps
from os import listdir, path
from itertools import combinations
import numpy as np

dataset_dir = 'Dataset/Push_to_Talk_Corpus/RefExp'

move_level_align = []
move_level_coherence = []
gamerun_level_align = []
gamerun_level_coherence = []

playermove_level_align = []
playermove_level_coherence = []
playergame_level_align = []
playergame_level_coherence = []

for dialogue in listdir(dataset_dir):
    dialogue = path.join(dataset_dir, dialogue)
    dialogue = refexps.load_data(dialogue)
    #move level
    dialogue_move = refexps.move_level(dialogue)
    for move in dialogue_move:
        refs = move[1]
        for obj in refs:
            refobj = refs[obj]
            sim_align = []
            sim_coherence = []
            for ref in refobj:
                sim = refexps.lex_sim(ref, refexps.get_previous(ref, refobj))
                if sim != 'undefined':
                    sim_align.append(sim)
            sim_align = np.mean(sim_align)
            comb_ref = combinations(refobj,2)
            for comb in comb_ref:
                if comb[0] != comb[1]:
                    sim = refexps.lex_sim(comb[0], comb[1])
                    if sim != 'undefined':
                        sim_coherence.append(sim)
            sim_coherence = np.mean(sim_coherence)
            if not np.isnan(sim_coherence) :
                move_level_coherence.append(sim_coherence)
            if not np.isnan(sim_align):
                move_level_align.append(sim_align)
                
print np.var(move_level_coherence), np.mean(move_level_coherence)
print np.var(move_level_align), np.mean(move_level_align)

0.0823660647305 0.206573284874
0.00817011681079 0.0242151584408
